<a href="https://colab.research.google.com/github/01koushal/T109-hacakathon-/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from fpdf import FPDF
import os

# STEP 1: Load and Prepare Data
data = pd.read_csv('water_data.csv')
data.dropna(inplace=True)
data['Gap'] = data['Reservoir_Storage'] - data['Water_Usage']
data['Risk'] = data['Gap'].apply(lambda x: 'High Risk' if x < 0 else 'Safe')

# STEP 2: Train the Model
X = data[['Year', 'Population', 'Rainfall']]
y = data['Water_Usage']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

# STEP 3: Forecast for 2030
future_input = pd.DataFrame({
    'Year': [2030],
    'Population': [1700000000],
    'Rainfall': [750]
})
predicted_usage = model.predict(future_input)[0]
projected_storage = 460
gap = projected_storage - predicted_usage
risk = 'High Risk' if gap < 0 else 'Safe'

# STEP 4: Forecast Multiple Future Years
years = list(range(2025, 2036))
future_df = pd.DataFrame({
    'Year': years,
    'Population': np.linspace(1400000000, 1750000000, len(years)).astype(int),
    'Rainfall': np.random.randint(700, 850, len(years))
})
future_df['Predicted_Usage'] = model.predict(future_df[['Year', 'Population', 'Rainfall']])
future_df['Projected_Storage'] = 460  # Constant for now
future_df['Gap'] = future_df['Projected_Storage'] - future_df['Predicted_Usage']
future_df['Risk'] = future_df['Gap'].apply(lambda x: 'High Risk' if x < 0 else 'Safe')

# STEP 5: Print Results
print("\n🔮 Forecast Results for 2030")
print("--------------------------------")
print(f"Predicted Water Usage     : {predicted_usage:.2f} units")
print(f"Projected Storage         : {projected_storage} units")
print(f"Gap (Storage - Usage)     : {gap:.2f} units")
print(f"⚠️ Risk Status             : {risk}")
print(f"📈 Model Accuracy (R²)     : {r2:.3f}")

print("\n📌 Recommendations:")
if risk == 'High Risk':
    print("- Increase reservoir capacity via infrastructure upgrades.")
    print("- Promote water conservation policies.")
    print("- Introduce efficient irrigation practices in agriculture.")
    print("- Plan for contingency during extreme climate conditions.")
else:
    print("- Maintain current water management strategy.")
    print("- Continue monitoring usage trends and population growth.")

# STEP 6: Charts
os.makedirs("outputs", exist_ok=True)

# Chart 1: Historical Data
plt.figure(figsize=(10, 6))
plt.plot(data['Year'], data['Water_Usage'], label='Water Usage', marker='o')
plt.plot(data['Year'], data['Reservoir_Storage'], label='Reservoir Storage', marker='x')
plt.fill_between(data['Year'], data['Water_Usage'], data['Reservoir_Storage'],
                 where=(data['Reservoir_Storage'] < data['Water_Usage']),
                 interpolate=True, color='red', alpha=0.3, label='Risk Zone')
plt.xlabel('Year')
plt.ylabel('Water Volume (units)')
plt.title('Historical Water Usage vs Storage')
plt.legend()
plt.grid(True)
chart1_path = "outputs/historical_chart.png"
plt.savefig(chart1_path)
plt.close()

# Chart 2: Forecast
plt.figure(figsize=(10, 6))
plt.plot(future_df['Year'], future_df['Predicted_Usage'], label='Predicted Usage', marker='o')
plt.plot(future_df['Year'], future_df['Projected_Storage'], label='Projected Storage', marker='x')
plt.fill_between(future_df['Year'], future_df['Predicted_Usage'], future_df['Projected_Storage'],
                 where=(future_df['Projected_Storage'] < future_df['Predicted_Usage']),
                 interpolate=True, color='orange', alpha=0.3, label='Risk Zone')
plt.xlabel('Year')
plt.ylabel('Water Volume (units)')
plt.title('Forecasted Water Usage vs Storage (2025–2035)')
plt.legend()
plt.grid(True)
chart2_path = "outputs/forecast_chart.png"
plt.savefig(chart2_path)
plt.close()

# STEP 7: Export to Excel
excel_path = "outputs/forecast_report.xlsx"
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    data.to_excel(writer, index=False, sheet_name='Historical Data')
    future_df.to_excel(writer, index=False, sheet_name='Forecast (2025–2035)')
    pd.DataFrame([{
        'Year': 2030,
        'Population': 1700000000,
        'Rainfall': 750,
        'Predicted_Usage': predicted_usage,
        'Projected_Storage': projected_storage,
        'Gap': gap,
        'Risk': risk,
        'R2_Score': r2
    }]).to_excel(writer, index=False, sheet_name='2030 Summary')
print(f"\n📊 Excel report saved to: {excel_path}")

# STEP 8: Export to PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=14)
pdf.cell(200, 10, txt="Forecast Report - Water Requirement & Storage (2030)", ln=True, align="C")

pdf.set_font("Arial", size=12)
pdf.ln(10)
pdf.cell(200, 10, txt=f"Forecast Year         : 2030", ln=True)
pdf.cell(200, 10, txt=f"Predicted Water Usage : {predicted_usage:.2f} units", ln=True)
pdf.cell(200, 10, txt=f"Projected Storage     : {projected_storage} units", ln=True)
pdf.cell(200, 10, txt=f"Gap (Storage - Usage) : {gap:.2f} units", ln=True)
pdf.cell(200, 10, txt=f"Risk Status           : {risk}", ln=True)
pdf.cell(200, 10, txt=f"Model Accuracy (R²)   : {r2:.3f}", ln=True)

pdf.ln(10)
pdf.multi_cell(0, 10, txt="""Recommendations:
- Increase reservoir capacity through infrastructure investments.
- Promote efficient water use in agriculture and urban sectors.
- Monitor population growth and future climate trends.
- Prepare contingency plans for extreme weather events.""")

pdf.ln(5)
pdf.image(chart1_path, x=10, w=180)
pdf.ln(5)
pdf.image(chart2_path, x=10, w=180)

pdf_path = "outputs/summary_report.pdf"
pdf.output(pdf_path)
print(f"📄 PDF report saved to: {pdf_path}")



🔮 Forecast Results for 2030
--------------------------------
Predicted Water Usage     : 676.87 units
Projected Storage         : 460 units
Gap (Storage - Usage)     : -216.87 units
⚠️ Risk Status             : High Risk
📈 Model Accuracy (R²)     : 1.000

📌 Recommendations:
- Increase reservoir capacity via infrastructure upgrades.
- Promote water conservation policies.
- Introduce efficient irrigation practices in agriculture.
- Plan for contingency during extreme climate conditions.

📊 Excel report saved to: outputs/forecast_report.xlsx
📄 PDF report saved to: outputs/summary_report.pdf
